In [2]:
import os
import datetime
from pathlib import Path
import glob
import itertools

folder = r"D:\20251114 HEK GV\GV\align"
transform_matrix = r"transform_matrix.json"
brightfield_transform_matrix = r"bf_transform_matrix.json"
separate_folders = True

timestamps = []
tiff_filelist = glob.glob(f"{folder}/*.tif*")
# output_path = os.path.join(folder, "aligned_output")
output_path = r"D:\20251114 HEK GV\GV\align\out"
print(output_path)


def roundTime(dt=None, roundTo=60):
    """Round a datetime object to any time lapse in seconds
    dt : datetime.datetime object, default now.
    roundTo : Closest number of seconds to round to, default 1 minute.
    Author: Thierry Husson 2012 - Use it as you want but don't blame me.
    """
    if dt == None:
        dt = datetime.datetime.now()
    seconds = (dt.replace(tzinfo=None) - dt.min).seconds
    rounding = (seconds + roundTo / 2) // roundTo * roundTo
    return dt + datetime.timedelta(0, rounding - seconds, -dt.microsecond)


def get_timestamp(filename):
    round_to_seconds = 5
    time = roundTime(
        datetime.datetime.fromtimestamp(os.path.getmtime(filename)), round_to_seconds
    )
    # print(time)
    return time


tiff_filelist = sorted(tiff_filelist, key=get_timestamp)
print("-")
wt_irm = itertools.groupby(tiff_filelist, get_timestamp)

for key, group in wt_irm:
    group = list(group)
    if len(group) == 3:
        for file in group:
            if "_IRM" in file:
                irm_file = file
            if "_WT" in file:
                wt_file = file
            if "_Bright" in file:
                bright_file = file

        assert "_IRM" in irm_file, "no IRM file"
        assert "_Bright" in bright_file, "no Bright file"
        assert "_WT" in wt_file, "no WT file"

        if separate_folders:
            subdirectory = os.path.join(
                output_path, os.path.splitext(os.path.basename(wt_file))[0]
            )
            print(subdirectory)
        else:
            subdirectory = output_path

        cmd_string = f'python video_aligner.py "{wt_file}" "{irm_file}" -bf "{bright_file}" -m "{transform_matrix}" -o "{subdirectory}" -bfm "{brightfield_transform_matrix}" '
        print(cmd_string)
        os.system(cmd_string)

    else:
        if len(group) > 3:
            raise f"{len(group)} files were found in the same group"


""" for file in tiff_filelist:
    time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
        second=0, microsecond=0
    )
    if time not in timestamps:
        timestamps.append(time)

for timestamp in timestamps:
    for file in tiff_filelist:
        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
            second=0, microsecond=0
        )
        if time == timestamp:
            if "WT" in file:
                wt_file = file
            else:
                irm_file = file 

    output_path = os.path.join(folder, Path(wt_file).stem + "_output")

    print(f"!python image_aligner.py {wt_file} {irm_file} -m '{transform_matrix}' -o {output_path}") """

D:\20251114 HEK GV\GV\align\out
-
D:\20251114 HEK GV\GV\align\out\20251114-085146_WT
python video_aligner.py "D:\20251114 HEK GV\GV\align\20251114-085146_WT.tiff" "D:\20251114 HEK GV\GV\align\20251114-085146_IRM.tiff" -bf "D:\20251114 HEK GV\GV\align\20251114-085146_Bright-field.tiff" -m "transform_matrix.json" -o "D:\20251114 HEK GV\GV\align\out\20251114-085146_WT" -bfm "bf_transform_matrix.json" 


' for file in tiff_filelist:\n    time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(\n        second=0, microsecond=0\n    )\n    if time not in timestamps:\n        timestamps.append(time)\n\nfor timestamp in timestamps:\n    for file in tiff_filelist:\n        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(\n            second=0, microsecond=0\n        )\n        if time == timestamp:\n            if "WT" in file:\n                wt_file = file\n            else:\n                irm_file = file \n\n    output_path = os.path.join(folder, Path(wt_file).stem + "_output")\n\n    print(f"!python image_aligner.py {wt_file} {irm_file} -m \'{transform_matrix}\' -o {output_path}") '